In [ ]:
from deep_orderbook.config import ReplayConfig, ShaperConfig
from deep_orderbook.shaper import iter_shapes_t2l
from deep_orderbook.visu import Visualizer
from deep_orderbook.strategy import Strategy

replay_conf = ReplayConfig(
    markets=["ETH-USD"],#, "BTC-USD", "ETH-BTC"],
    data_dir='/media/photoDS216/crypto/',
    date_regexp='2025-02*',
    max_samples=-1,
    every="100ms",
)
shaper_config = ShaperConfig(
    only_full_arrays=False,
    view_bips = 5,
    num_side_lvl = 8,
    look_ahead = 32,
    look_ahead_side_bips = 5,
    look_ahead_side_width = 4,
    rolling_window_size = 256,
    window_stride = 16,
)

vis = Visualizer()
strategy = Strategy(threshold=0.2)
async def rolling_plot(config: ReplayConfig):
    async for books_array, level_prox, pxar in iter_shapes_t2l(
        replay_config=config, #.but(every="200ms"),
        shaper_config=shaper_config, #.but(view_bips=4, look_ahead_side_bips=1)
    ):
         gt_pnl, pos, gt_up_prox, gt_down_prox = strategy.compute_pnl(pxar, level_prox)
         vis.update(
             books_z_data=books_array, level_reach_z_data=level_prox, bidask=pxar,
             gt_pnl=gt_pnl, positions=pos,
         )

await rolling_plot(config=replay_conf.but(
        # max_samples=1000, 
        # skip_until_time="23:15",
    )
)